In [46]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [120]:
def generated_spatial_features_dataframe(path):
    key_value_link_list = []
    with open(path,'r') as f:
        urls = f.read()
        urls = urls.split('\n')
    
    for url in urls[:3]:
        key_value_link_list = key_value_link_list + fetch_group(url)
    
    return pd.DataFrame(key_value_link_list,columns=['key','value','osm_wiki_url'])
    
    
def fetch_group(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tmp_list = []

    for tbody in soup.find_all('tbody'):
        for tr in tbody.find_all('tr'):
            for td in tr.find_all('td'):
                found_key = None
                try:
                    title = td.a['title']
                    if 'Tag:' in title:
                        key,value = title[4:].split('=')
                        tmp_list.append((key,value,f"https://wiki.openstreetmap.org{td.a['href']}"))
                except:
                    pass
    
    return tmp_list

def get_wikidata(url):
    wikidata_url = None
    
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        wikidata_url = soup.find('tr', {'class' : 'd_wikidata content'}).a['href']
    
    except Exception as e:
        wikidata_url = e
    
    return wikidata_url

In [117]:
kv_df = generated_spatial_features_dataframe('./osm_groups.txt')

In [118]:
small_df = kv_df.loc[:5].copy()

In [121]:
small_df['wikidata_url'] = small_df['osm_wiki_url'].apply(get_wikidata)


,key,value,osm_wiki_url,wikidata_url
0,amenity,bar,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://www.wikidata.org/wiki/Q187456
1,amenity,bbq,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://www.wikidata.org/wiki/Q853185
2,amenity,biergarten,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://www.wikidata.org/wiki/Q857909
3,amenity,cafe,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://www.wikidata.org/wiki/Q30022
4,amenity,drinking water,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://query.wikidata.org/#SELECT%20%3Fitem%2...
5,amenity,fast food,https://wiki.openstreetmap.org/wiki/Tag:amenit...,https://www.wikidata.org/wiki/Q1751429


In [124]:
small_df.loc[4]['osm_wiki_url']

'https://wiki.openstreetmap.org/wiki/Tag:amenity%3Ddrinking_water'

In [125]:
small_df.loc[0]['osm_wiki_url']

'https://wiki.openstreetmap.org/wiki/Tag:amenity%3Dbar'

In [98]:
print(response.text)


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Tag:amenity=bar - OpenStreetMap Wiki</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"aee1c9ea0703daaf525bd66f","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Tag:amenity=bar","wgTitle":"Tag:amenity=bar","wgCurRevisionId":2077172,"wgRevisionId":2077172,"wgArticleId":35187,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages unavailable in Dutch","Item with no description in language UK","Item with no description in language FI","Item with no description in language NL","Item with no des

In [108]:
# soup.find(id="link3")
# <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
soup.find('tr', {'class' : 'd_wikidata content'}).a['href']


'https://www.wikidata.org/wiki/Q187456'

In [74]:
key_value_link = []

for tbody in soup.find_all('tbody'):
    for tr in tbody.find_all('tr'):
        for td in tr.find_all('td'):
            found_key = None
            try:
                title = td.a['title']
                if 'Tag:' in title:
                    key,value = title[4:].split('=')
                    key_value_link.append((key,value,f"https://wiki.openstreetmap.org{td.a['href']}"))
            except:
                pass
key_value_link

[('shop', 'alcohol', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dalcohol'),
 ('shop', 'bakery', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dbakery'),
 ('shop',
  'beverages',
  'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dbeverages'),
 ('shop',
  'brewing supplies',
  'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dbrewing_supplies'),
 ('shop', 'butcher', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dbutcher'),
 ('shop', 'cheese', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dcheese'),
 ('shop',
  'chocolate',
  'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dchocolate'),
 ('shop', 'coffee', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dcoffee'),
 ('shop',
  'confectionery',
  'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dconfectionery'),
 ('shop',
  'convenience',
  'https://wiki.openstreetmap.org/wiki/Tag:shop%3Dconvenience'),
 ('shop', 'deli', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3Ddeli'),
 ('shop', 'dairy', 'https://wiki.openstreetmap.org/wiki/Tag:shop%3

In [72]:
for tr in soup.find_all('tbody')[2].find_all('tr'):
    print(tr)

<tr style="background-color:#F8F4C2">
<th style="width:70px">Key
</th>
<th style="width:100px">Value
</th>
<th style="width:70px">Element
</th>
<th>Comment
</th>
<th style="width:100px">Rendering on default layer (osm-<a href="/wiki/CartoCSS" title="CartoCSS">carto</a>)
</th>
<th style="width:100px">Photo
</th></tr>
<tr>
<th colspan="7" style="border-bottom: 3px double;">
<h4><span id="Food,_beverages"></span><span class="mw-headline" id="Food.2C_beverages">Food, beverages</span></h4>
</th></tr>
<tr>
<td><a class="mw-selflink selflink"> shop</a>
</td>
<td><a href="/wiki/Tag:shop%3Dalcohol" title="Tag:shop=alcohol"> alcohol</a>
</td>
<td><a href="/wiki/Node" title="node"><img alt="node" decoding="async" height="20" src="/w/images/thumb/7/76/Osm_element_node.svg/20px-Osm_element_node.svg.png" srcset="/w/images/thumb/7/76/Osm_element_node.svg/30px-Osm_element_node.svg.png 1.5x, /w/images/thumb/7/76/Osm_element_node.svg/40px-Osm_element_node.svg.png 2x" style="vertical-align: text-bottom" 